In [9]:
!pip install duckdb
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/9b/35/74442388c6cf008882d4d4bdfc4109be87e9b8b7ccd097ad1e7f006e2e95/pandas-2.3.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/c7/b0/003792df09decd6849a5e39c28b513c06e84436a54440380862b5aeff25d/tzdata-2025.3-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 4.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 kB 1.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
Creating DuckDB Database

In [2]:
import duckdb

conn = duckdb.connect('my_database.db')

In [3]:
conn

we can also use in-memory database with the below command, but when we shut down the system, the data will be lost.

its always advisable to use persistant memory.

In [4]:
conn_memory = duckdb.connect(':memory:')

In [5]:
conn_memory

Loading Data into DuckDB

In [6]:
conn.execute(
    '''
CREATE TABLE employees(
id INTEGER PRIMARY KEY,
name VARCHAR,
age INTEGER,
department VARCHAR
)
    '''
)

To verify the tables, we can use SHOW command.

Execute query runs the SQL query and resturns the DuckDB result set -> we need to convert to data frame to view the result.

In [12]:
conn.execute('SHOW TABLES').df()

,name
0,employees


Inserting a record

In [14]:
conn.execute(
    '''
INSERT INTO employees VALUES

(1, 'thanga', 31, 'Data Lead'),
(2, 'mayil', 29, 'Dev Ops'),
(3, 'kumar', 35, 'Operations'),
(4, 'saavan', 36, 'CEO')


'''
)

Querying the tables

In [17]:
conn.execute(
    '''
SELECT * FROM employees
'''
).df()

,id,name,age,department
0,1,thanga,31,Data Lead
1,2,mayil,29,Dev Ops
2,3,kumar,35,Operations
3,4,saavan,36,CEO


Aggregation


lets count the no of employees in each department using the COUNT and GROUP BY 

In [18]:
conn.execute(
    '''
SELECT department, count(*) AS employee_count
FROM employees
GROUP BY department
'''
).df()

,department,employee_count
0,CEO,1
1,Operations,1
2,Data Lead,1
3,Dev Ops,1


Calculating the average age of the employees

In [23]:
conn.execute(
    '''
select AVG(age) from employees
'''
).df()

,avg(age)
0,32.75


Finding out the oldest / senior most employees in each department

In [25]:

conn.execute(
    '''
select department, max(age) as seniority
from employees
group by department
'''
).df()

,department,seniority
0,Dev Ops,29
1,CEO,36
2,Data Lead,31
3,Operations,35



Joining the tables


First table - Orders

In [26]:
conn.execute(
    '''
create table orders (
order_id INTEGER,
customer_id INTEGER,
amount FLOAT)'''
)

In [27]:
conn.execute(
    '''
insert into orders values
(1,1,100.0),
(2,2,300),
(3,1,340.0)'''
)


Second Table -  Customers


In [28]:
conn.execute(
    '''
create table customers(
customer_id INTEGER,
name VARCHAR)'''
)

In [29]:
conn.execute(
    '''
insert into customers values (1, 'Alice'), (2, 'Bob')
'''
)

In [31]:
conn.execute('''
select * from orders
             ''').df()

,order_id,customer_id,amount
0,1,1,100.0
1,2,2,300.0
2,3,1,340.0


In [32]:
conn.execute('''
select * from customers
             ''').df()

,customer_id,name
0,1,Alice
1,2,Bob


Joining two tables

In [39]:
conn.execute(
    '''
select customers.customer_id,
customers.name,
orders.customer_id as from_orders_table, 
orders.amount from orders join customers
on orders.customer_id = customers.customer_id
order by 
customers.customer_id
'''
).df()

,customer_id,name,from_orders_table,amount
0,1,Alice,1,100.0
1,1,Alice,1,340.0
2,2,Bob,2,300.0


In [48]:
conn.execute(
    '''
select SUM(orders.amount) as total,
customers.name,
customers.customer_id  from orders join customers
on orders.customer_id = customers.customer_id
group by
customers.customer_id,
customers.name
order by 
customers.customer_id
'''
).df()

,total,name,customer_id
0,440.0,Alice,1
1,300.0,Bob,2


Loading from data frames.

Loading flights dataset

In [49]:
import pandas as pd

In [52]:
conn.execute(
    '''
    show tables
    '''
).df()

,name
0,customers
1,employees
2,orders



%timeit is the magic command to measure the time it takes to load the csv file into a data frame.



In [55]:
# %timeit df = pd.read_csv('flights.csv')
